In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time
import redis
from flask import current_app

In [2]:
data = pd.read_csv('./ml_3000.csv')

In [3]:
data.head(20)

,Text,G/L
0,Investment Increase,45604
1,Investment Increase,45604
2,INV,113100
3,CHECK,113101
4,Hilfs Loehne,421000
5,Salaries Wages,430000
6,Mehrarbeit Gehlter,431000
7,Kraftfahrzeug Steuer,464000
8,Basic rent own use,470000
9,Own use OC flat rate,470000


In [4]:
tfidf = TfidfVectorizer(ngram_range=(1,3), use_idf=True)

In [5]:
tfidf_matrix = tfidf.fit_transform(data['Text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

for idx, row in data.iterrows():
            similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
            similar_items = [(cosine_similarities[idx][i], data['G/L'][i])
                             for i in similar_indices]

            flattened = sum(similar_items[1:],())

In [6]:
similarities = list()
item_id = list()
for i in range(0,len(flattened),2): 
    similarities.append(flattened[i]) # Collecting similarity scores and 
    item_id.append(flattened[i-1])

In [7]:
print("Similar Product Id's:",similar_items)

Similar Product Id's: [(1.0, 888900), (1.0, 888900), (1.0, 888900), (1.0, 888900), (1.0, 888900), (1.0, 888900), (1.0, 888900), (0.8054370400407993, 470000), (0.8054370400407993, 470000), (0.8054370400407993, 470000), (0.8054370400407993, 470000), (0.8054370400407993, 470000), (0.8054370400407993, 470000), (0.357845024289353, 888900), (0.357845024289353, 888900), (0.357845024289353, 888900), (0.357845024289353, 888900), (0.357845024289353, 888900), (0.357845024289353, 888900), (0.14528626927585059, 470000), (0.14528626927585059, 470000), (0.14528626927585059, 470000), (0.14528626927585059, 470000), (0.14528626927585059, 470000), (0.14528626927585059, 470000), (0.12101635359398295, 888900), (0.12101635359398295, 888900), (0.12101635359398295, 888900), (0.12101635359398295, 888900), (0.12101635359398295, 888900), (0.059765509151178287, 841070), (0.059765509151178287, 841070), (0.059765509151178287, 841070), (0.059765509151178287, 841070), (0.059765509151178287, 841070), (0.05976550915117

In [8]:
sol = pd.DataFrame({'G/L':item_id,'similarities':similarities})
predictions = sol.merge(data,how='inner',on='G/L')[:10] # using merge to get description

In [9]:
predictions

,G/L,similarities,Text
0,45604,1.0,Investment Increase
1,45604,1.0,Investment Increase
2,45604,0.0,Investment Increase
3,45604,0.0,Investment Increase
4,888900,1.0,Basic rent transfer
5,888900,1.0,OC flat rate trsfr own use
6,888900,1.0,Transfer basic rent own use
7,888900,1.0,Basic rent transfer
8,888900,1.0,OC flat rate trsfr own use
9,888900,1.0,Transfer basic rent own use


In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [12]:
le.fit(data['G/L'])

LabelEncoder()

In [17]:
Product_Class=le.transform(data['G/L'])
print(Product_Class)

[ 0  0  1  2  3  4  5  6  7  7  7  7  7  7  7  7  7  7  7  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 22 22 22 22 22 23 23 23 23 23 23 23 23 23 23
 23 23 23 23 23 23 23 23]


In [21]:
sol = pd.DataFrame({'G/L':item_id,'similarities':similarities})

In [22]:
sol

,G/L,similarities
0,45604,1.000000
1,888900,1.000000
2,888900,1.000000
3,888900,1.000000
4,888900,1.000000
5,888900,1.000000
6,888900,0.805437
7,470000,0.805437
8,470000,0.805437
9,470000,0.805437


In [23]:
predictions = sol.merge(data,how='inner',on='G/L')[:10] # using merge to get description

## Final Similarities between the Product IDs and Descriptions

In [26]:
predictions

,G/L,similarities,Text
0,45604,1.0,Investment Increase
1,45604,1.0,Investment Increase
2,45604,0.0,Investment Increase
3,45604,0.0,Investment Increase
4,888900,1.0,Basic rent transfer
5,888900,1.0,OC flat rate trsfr own use
6,888900,1.0,Transfer basic rent own use
7,888900,1.0,Basic rent transfer
8,888900,1.0,OC flat rate trsfr own use
9,888900,1.0,Transfer basic rent own use
